<a href="https://colab.research.google.com/github/Taaniya/semantic-search-for-info-retrieval/blob/main/semantic_search_evaluation_with_ranx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install ranx

In [7]:
! ls ./drive/MyDrive/Colab\ Notebooks/NLP_exploration/QA_on_docs/

Explore_BERT.ipynb
Exploring_Flan_T5_with_langchain.ipynb
Finetuning_bert_for_semantic_search_with_SBERT_approach.ipynb
knowledge_doc.pdf
qa_on_docs_with_flan_t5.ipynb
qrels_v2.xlsx
run.xlsx
Semantic_search_comparison_with_FAISS_standalone_vs_via_Langchain.ipynb
sentence_embeddings_with_BERT.ipynb
Untitled0.ipynb


In [13]:
import pandas as pd
import json

from ranx.metrics.f1 import _f1_parallel, _f1, f1
from ranx import Qrels, Run, evaluate


In [14]:
qrels_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/QA_on_docs/qrels_v2.xlsx"
preds_run_path = "./drive/MyDrive/Colab Notebooks/NLP_exploration/QA_on_docs/run.xlsx"

In [15]:
qrels_df = pd.read_excel(qrels_path)
run_df = pd.read_excel(preds_run_path)

In [16]:
# drop rows where there is no doc_id for a query
run_df = run_df.dropna(subset=['doc_id'])

# cast doc_id to int
run_df['doc_id'] = run_df['doc_id'].astype('int')

qrels_df['doc_id'] = 'd_' + qrels_df['doc_id'].astype(str)
run_df['doc_id'] = 'd_' + run_df['doc_id'].astype(str)

# cast q_id columns to string type
qrels_df['q_id'] = qrels_df['q_id'].astype(str)
run_df['q_id'] = run_df['q_id'].astype(str)

<ipython-input-16-9ad9bd6a9bc7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_df['doc_id'] = run_df['doc_id'].astype('int')


In [17]:
# Create qrels object
qrels = Qrels.from_df( df=qrels_df,  q_id_col="q_id", doc_id_col="doc_id")

# Create run object
run = Run.from_df(df=run_df,
                  q_id_col="q_id",
                  doc_id_col="doc_id",
                  score_col="cosine_similarity")

In [26]:
# perform evaluation
score_dict = evaluate(qrels, run, ["mrr", "mrr@1", "mrr@2", "recall@1", "recall@2", "precision@1", "f1@1", "hits@1"])

for k,v in score_dict.items():
  print(f"{k}: {v}")

mrr: 0.9629629629629629
mrr@1: 0.9444444444444444
mrr@2: 0.9444444444444444
recall@1: 0.9444444444444444
recall@2: 0.9444444444444444
precision@1: 0.9444444444444444
f1@1: 0.7777777777777778
hits@1: 0.9444444444444444


In [19]:
# pretty print scores
print(json.dumps(run.scores, indent=4))

{
    "mrr": {
        "0": 1.0,
        "1": 1.0,
        "10": 1.0,
        "11": 1.0,
        "12": 1.0,
        "13": 1.0,
        "14": 1.0,
        "15": 1.0,
        "16": 1.0,
        "17": 1.0,
        "2": 1.0,
        "3": 1.0,
        "4": 1.0,
        "5": 1.0,
        "6": 0.3333333333333333,
        "7": 1.0,
        "8": 1.0,
        "9": 1.0
    },
    "mrr@1": {
        "0": 1.0,
        "1": 1.0,
        "10": 1.0,
        "11": 1.0,
        "12": 1.0,
        "13": 1.0,
        "14": 1.0,
        "15": 1.0,
        "16": 1.0,
        "17": 1.0,
        "2": 1.0,
        "3": 1.0,
        "4": 1.0,
        "5": 1.0,
        "6": 0.0,
        "7": 1.0,
        "8": 1.0,
        "9": 1.0
    },
    "mrr@2": {
        "0": 1.0,
        "1": 1.0,
        "10": 1.0,
        "11": 1.0,
        "12": 1.0,
        "13": 1.0,
        "14": 1.0,
        "15": 1.0,
        "16": 1.0,
        "17": 1.0,
        "2": 1.0,
        "3": 1.0,
        "4": 1.0,
        "5": 1.0,
    

In [20]:
qrels_df.count()

q_id      18
doc_id    18
score     18
dtype: int64

In [21]:
run_df.count()

q_id                 29
doc_id               29
cosine_similarity    29
dtype: int64

In [22]:
# convert qrels and run object to numba typed list to call functions f1, _f1, _f1_parallel

_qrels = qrels.to_typed_list()
_run = run.to_typed_list()

In [28]:
print(f1(_qrels, _run, 1, 1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]


In [29]:
print(_f1_parallel(_qrels, _run, 1 , 1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]


In [30]:
scores = []

for i in range(len(_qrels)):
  try:
      scores.append(_f1(_qrels[i], _run[i], 1 , 1))
  except Exception as error:
    # print division by zero exception
    print(f" {i} An exception occurred:", error)
    scores.append(0)
    continue

 14 An exception occurred: division by zero


In [31]:
scores

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0,
 1.0,
 1.0,
 1.0]